# Limitations

*Please read and understand these carefully*

While the benefit of an openly available, functional and partially
validated model of the European transmission system is high, many
approximations have been made due to missing data. In this section we
summarise the limitations of the dataset, both as a warning to the
user and as an encouragement to assist in improving the approximations.

The grid data is based on a map of the ENTSO-E area
that is known to contain small distortions to improve
readability. Since the exact impedances of the lines are unknown,
approximations based on line lengths and standard line parameters
ignore specific conductoring choices for particular lines. There is no
openly available data on busbar configurations, switch locations,
transformers or reactive power compensation assets.

Using Voronoi cells to aggregate load and generator data to
transmission network substations ignores the topology of the
underlying distribution network, meaning that assets may be connected
to the wrong substation. Assumptions have been made about the
distribution of load in each country proportional to population and
GDP that may not reflect local circumstances. Openly available data on
load time series may not correspond to the true vertical load and is not
spatially disaggregated; assuming, as we have done, that the load time
series shape is the same at each node within each country ignores local
differences.

Wind, solar and small hydro, geothermal, marine and biomass power
plants are excluded from the dataset because of a lack of data
availability in many countries. Approximate distributions of wind and
solar plants in each country can be generated that are proportional to
the capacity factor at each location.

The database of hydro-electric power plants does not include
plant-specific energy storage information, so that blanket values
based on country storage totals have been used. Inflow time series are
based on country-wide approximations, ignoring local topography and
basin drainage; in principle a full hydrological model should be used.

Border connections and power flows to Russia, Belarus, Ukraine, Turkey
and Morocco have not been taken into account; islands which are not
connected to the main European system, such as Malta, Crete and
Cyprus, are also excluded from the model.


# Initialization

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('white')

import pandas as pd
import numpy as np
import geopandas as gpd

import scipy as sp, scipy.sparse

import shapely.wkt
from shapely.geometry import Point
import shapely.prepared

import os
from itertools import chain, count, ifilter
from six import itervalues, iteritems
from operator import attrgetter

import xarray as xr

In [ ]:
import pypsa

from vresutils import (shapes as vshapes, plot as vplot, landuse as vlanduse,
                       costdata2 as vcostdata2, hydro as vhydro)
from vresutils.graph import voronoi_partition_pts, BreadthFirstLevels
from vresutils.costdata import annuity, USD2013_to_EUR2013

import powerplantmatching as pm

import atlite

# Configuration

In [ ]:
countries = ['AL', 'AT', 'BA', 'BE', 'BG', 'CH', 'CZ', 'DE', 'DK', 'EE', 'ES',
             'FI', 'FR', 'GB', 'GR', 'HR', 'HU', 'IE', 'IT', 'LT', 'LU', 'LV',
             'ME', 'MK', 'NL', 'NO', 'PL', 'PT', 'RO', 'RS', 'SE', 'SI', 'SK']

# Network topology

## Data files

### Buses

In [ ]:
buses = (pd.read_csv("data/buses.csv", quotechar="'", true_values='t', false_values='f', dtype=dict(bus_id="str"))
         .set_index("bus_id")
         .rename(columns=dict(voltage='v_nom')))

In [ ]:
buses_pts = buses.pop('geometry').map(shapely.wkt.loads)
buses['x'] = buses_pts.map(attrgetter('x'))
buses['y'] = buses_pts.map(attrgetter('y'))

In [ ]:
buses['carrier'] = buses.pop('dc').map({True: 'DC', False: 'AC'})

### Transformers and Converters

In [ ]:
transformers = (pd.read_csv("data/transformers.csv", quotechar="'", true_values='t', false_values='f',
                            dtype=dict(transformer_id='str', src_bus_id='str', dst_bus_id='str'))
                .set_index('transformer_id')
                .rename(columns=dict(src_bus_id='bus0', dst_bus_id='bus1', voltage='v_nom'))
                .drop(['symbol'], axis=1))

In [ ]:
# src_dc, dst_dc, src_voltage and dst_voltage contain redundant information,
# as can easily be verified

assert ((transformers.src_dc == (transformers.bus0.map(buses.carrier) == 'DC'))
        | transformers.src_dc.isnull()).all()
assert ((transformers.dst_dc == (transformers.bus1.map(buses.carrier) == 'DC'))
        | transformers.dst_dc.isnull()).all()
assert ((transformers.src_voltage == transformers.bus0.map(buses.v_nom))
        | transformers.src_voltage.isnull()).all()
assert ((transformers.dst_voltage == transformers.bus1.map(buses.v_nom))
        | transformers.dst_voltage.isnull()).all()

transformers.drop(['src_dc', 'dst_dc', 'src_voltage', 'dst_voltage'],
                  inplace=True, axis=1)

There are transformers and converters in the dataset. Converters are those, which connect an AC to a DC bus.

In [ ]:
same_carrier = transformers.bus1.map(buses.carrier) == transformers.bus0.map(buses.carrier)
converters = transformers.loc[~ same_carrier]
transformers = transformers.loc[same_carrier]
del same_carrier

### AC and DC lines

In [ ]:
lines = (pd.read_csv("data/links.csv", quotechar="'", true_values='t', false_values='f',
                     dtype=dict(link_id='str', src_bus_id='str', dst_bus_id='str'))
         .set_index('link_id')
         .rename(columns=dict(src_bus_id='bus0', dst_bus_id='bus1', voltage='v_nom')))

In [ ]:
lines = lines.assign(length=lines['length_m'] / 1e3).drop('length_m', axis=1)

In [ ]:
dclines = lines.query('dc').drop('dc', axis=1)
aclines = lines.query('~ dc').drop('dc', axis=1)

## Reduce dataset

### Remove duplicate lines

In [ ]:
line_dups = pd.read_csv('data/line-dups.csv', dtype='str', skiprows=11)['line_id']
aclines.drop(line_dups, inplace=True)

In [ ]:
dclines = pd.DataFrame(dclines[~pd.DataFrame(dict(lat1=dclines.bus0.map(buses.y), lon1=dclines.bus0.map(buses.x),
                                                  lat2=dclines.bus1.map(buses.y), lon2=dclines.bus1.map(buses.x))).duplicated()])

### Remove all but 220kV, 300kV and 380kV voltage levels

In [ ]:
voltage_levels_to_remove = [110.0,
                            150.0,
                            132.0,
                            750.0]

In [ ]:
buses_with_v_nom_to_keep_b = ~ buses.v_nom.isin(voltage_levels_to_remove)

In [ ]:
transformers_with_v_nom_to_keep_b = transformers.bus0.map(buses_with_v_nom_to_keep_b) & transformers.bus1.map(buses_with_v_nom_to_keep_b)
converters_with_v_nom_to_keep_b = converters.bus0.map(buses_with_v_nom_to_keep_b) & converters.bus1.map(buses_with_v_nom_to_keep_b)
aclines_with_v_nom_to_keep_b = aclines.bus0.map(buses_with_v_nom_to_keep_b) & aclines.bus1.map(buses_with_v_nom_to_keep_b)
dclines_with_v_nom_to_keep_b = dclines.bus0.map(buses_with_v_nom_to_keep_b) & dclines.bus1.map(buses_with_v_nom_to_keep_b)

### Remove everything not in any of the configured countries

In [ ]:
europe_shape = vshapes.country_cover(countries)
europe_shape_exterior = shapely.geometry.Polygon(shell=europe_shape.exterior) # no holes

In [ ]:
vplot.shapes(europe_shape_exterior, colour='blue').set_zorder(-1)
vplot.draw_basemap()

In [ ]:
import shapely.prepared
europe_shape_exterior_prepped = shapely.prepared.prep(europe_shape_exterior)
buses_in_europe_b = buses_pts.map(lambda p: europe_shape_exterior_prepped.contains(p))
del europe_shape_exterior_prepped
#buses = buses.loc[buses_in_europe]

In [ ]:
transformers_in_europe_b = transformers.bus0.map(buses_in_europe_b) | transformers.bus1.map(buses_in_europe_b)
converters_in_europe_b = converters.bus0.map(buses_in_europe_b) | converters.bus1.map(buses_in_europe_b)
aclines_in_europe_b = aclines.bus0.map(buses_in_europe_b) | aclines.bus1.map(buses_in_europe_b)
dclines_in_europe_b = dclines.bus0.map(buses_in_europe_b) | dclines.bus1.map(buses_in_europe_b)

In [ ]:
from functools import reduce

buses_in_europe_b |= pd.Series(
    buses.index.isin(reduce(set.union,
                            (x[i]
                             for x in (converters.loc[converters_in_europe_b],
                                       transformers.loc[transformers_in_europe_b],
                                       aclines.loc[aclines_in_europe_b],
                                       dclines.loc[dclines_in_europe_b])
                             for i in ('bus0', 'bus1')),
                            set())),
    index=buses.index)

### Perform prepared removals

In [ ]:
buses = pd.DataFrame(buses.loc[buses_in_europe_b & buses_with_v_nom_to_keep_b])
converters = pd.DataFrame(converters.loc[converters_in_europe_b & converters_with_v_nom_to_keep_b])
transformers = pd.DataFrame(transformers.loc[transformers_in_europe_b & transformers_with_v_nom_to_keep_b])
aclines = pd.DataFrame(aclines.loc[aclines_in_europe_b & aclines_with_v_nom_to_keep_b])
dclines = pd.DataFrame(dclines.loc[dclines_in_europe_b & dclines_with_v_nom_to_keep_b])

## Adjust parameters

In [ ]:
def find_closest_bus(pos):
    if (not hasattr(find_closest_bus, 'kdtree')) or len(find_closest_bus.kdtree.data) != len(buses.index):
        find_closest_bus.kdtree = sp.spatial.cKDTree(buses.loc[:,["x", "y"]].values)
    return buses.index[find_closest_bus.kdtree.query(pos)[1]]

### Transformers

In [ ]:
transformers = transformers.append(
    pd.DataFrame([(str(transformers.index.astype(int).max()+1),
                   find_closest_bus((22.6, 53.9)),
                   find_closest_bus((22.5, 53.9)))],
                 columns=['index', 'bus0', 'bus1']).set_index('index')
)

In [ ]:
transformers["x"] = 0.1
transformers["s_nom"] = 2000
transformers['type'] = ""

### Converters + DC-Lines ->Links

In [ ]:
converters = converters.append(
    pd.DataFrame([
         (i, find_closest_bus(pos1), find_closest_bus(pos2))
         for i, (pos1, pos2) in enumerate([((6.8, 59.6), (6.65, 59.5)),
                                           ((16.7, 57.6), (16.7, 57.4)),
                                           ((1.3, 51.2), (1.1, 51.3))],
                                          start=converters.index.astype(int).max()+1)
    ], columns=['index', 'bus0', 'bus1']).set_index('index')
)

In [ ]:
converters = converters.groupby('bus0', as_index=False).apply(
    lambda x: x.iloc[0] if len(x) == 1 else x.loc[x['bus1'].map(buses['v_nom']).idxmax()]
)

In [ ]:
converters_ac_to_dc_b = (converters.bus0.map(buses['carrier']) == 'AC') & (converters.bus1.map(buses['carrier']) == 'DC')
dc_to_ac_map = (pd.Series(converters.loc[converters_ac_to_dc_b, 'bus0'].values, index=converters.loc[converters_ac_to_dc_b, 'bus1'].values)
                .append(pd.Series(converters.loc[~converters_ac_to_dc_b, 'bus1'].values, index=converters.loc[~converters_ac_to_dc_b, 'bus0'].values)))
missing_dc_buses_i = buses[buses['carrier'] == 'DC'].index.difference(dc_to_ac_map.index)
dc_to_ac_map = dc_to_ac_map.append(pd.Series(missing_dc_buses_i, index=missing_dc_buses_i))

Drop unneeded buses

In [ ]:
buses.drop(dc_to_ac_map.index[dc_to_ac_map.index.values != dc_to_ac_map.values], inplace=True)

Rewire dclines

In [ ]:
dclines['bus0'] = dclines['bus0'].map(dc_to_ac_map)
dclines['bus1'] = dclines['bus1'].map(dc_to_ac_map)

In [ ]:
dclines = dclines.append(
    pd.DataFrame([(str(dclines.index.astype(int).max()+1),
                   500,
                   find_closest_bus((23.9, 54.4)),
                   find_closest_bus((24., 54.4)),
                   vshapes.haversine([(23.9, 54.4), (24., 54.4)]))],
                 columns=['index', 'p_nom', 'bus0', 'bus1', 'length']).set_index('index')
)

In [ ]:
dclines.drop('v_nom', axis=1, inplace=True)

### DC lines

DC line capacities from [List of HVDC Projects on Wikipedia](https://en.wikipedia.org/wiki/List_of_HVDC_projects)

In [ ]:
dclines['p_nom'] = pd.read_csv('data/dclines_p-nom.csv', names=['name', 'p_nom'], dtype={'name': str, 'p_nom': float}, skiprows=1).set_index('name')
dclines['p_min_pu'] = -1.

### AC-Lines

#### Split AC-Lines with multiple voltages

In [ ]:
def namer(string, start=0): return (string.format(x) for x in count(start=start))
busname = namer("M{:02}")
trafoname = namer("M{:02}")
linename = namer("M{:02}")

In [ ]:
def find_or_add_lower_v_nom_bus(bus, v_nom):
    candidates = transformers.loc[(transformers.bus1 == bus) & (transformers.bus0.map(buses.v_nom) == v_nom), 'bus0']
    if len(candidates):
        return candidates.iloc[0]
    new_bus = next(busname)
    buses.loc[new_bus] = pd.Series({'v_nom': v_nom, 'symbol': 'joint', 'carrier': 'AC',
                                    'x': buses.at[bus, 'x'], 'y': buses.at[bus, 'y'],
                                    'under_construction': buses.at[bus, 'under_construction']})
    transformers.loc[next(trafoname)] = pd.Series({'bus0': new_bus, 'bus1': bus, 'x': 0.1, 's_nom': 2000, 'type': ""})
    return new_bus

In [ ]:
voltage_levels = aclines.v_nom.unique()

for line in aclines.tags.str.extract(r'"text_"=>"\(?(\d+)\+(\d+).*?"', expand=True).dropna().itertuples():
    v_nom = int(line._2)
    if aclines.at[line.Index, 'circuits'] > 1:
        aclines.at[line.Index, 'circuits'] -= 1

    if v_nom in voltage_levels:
        bus0 = find_or_add_lower_v_nom_bus(aclines.at[line.Index, 'bus0'], v_nom)
        bus1 = find_or_add_lower_v_nom_bus(aclines.at[line.Index, 'bus1'], v_nom)
        aclines.loc[next(linename)] = pd.Series(dict(chain(iteritems({'bus0': bus0, 'bus1': bus1, 'v_nom': v_nom, 'circuits': 1}),
                                                           iteritems({k: aclines.at[line.Index, k]
                                                                      for k in {'underground', 'under_construction',
                                                                                'tags', 'geometry', 'length'}}))))

#### Electrical parameters

Apply standard line-types from https://pypsa.org/doc/components.html#line-types

In [ ]:
# for v_nom, linetype in [(220., "Al/St 240/40 2-bundle 220.0"),
#                         (300., "Al/St 240/40 3-bundle 300.0"),
#                         (380., "Al/St 240/40 4-bundle 380.0")]:
#     aclines.loc[aclines["v_nom"] == v_nom, 'type'] = linetype

In [ ]:
from collections import namedtuple

LineParam = namedtuple("LineParam", ["v_nom", "wires", "r", "x", "c", "i"])
for p in (LineParam(v_nom=220., wires=2.0, r=0.06, x=0.301, c=12.5, i=1.29),
          LineParam(v_nom=380., wires=4.0, r=0.03, x=0.246, c=13.8, i=2.58),
          LineParam(v_nom=300., wires=3.0, r=0.04, x=0.265, c=13.2, i=1.935)):
    ls = aclines["v_nom"] == p.v_nom
    length = aclines.loc[ls, "length"]
    circuits = aclines.loc[ls, "circuits"]
    aclines.loc[ls, "r"] = length * p.r / circuits
    aclines.loc[ls, "x"] = length * p.x / circuits
    aclines.loc[ls, "b"] = length * (2*np.pi*50*1e-9) * p.c * circuits
    aclines.loc[ls, "s_nom"] = np.sqrt(3.) * p.v_nom * p.i * circuits

In [ ]:
aclines['type'] = ""

## PyPSA model

In [ ]:
network = pypsa.Network()
network.name = "PyPSA-Europe"
network

In [ ]:
pypsa.io.import_components_from_dataframe(network, buses, "Bus")
pypsa.io.import_components_from_dataframe(network, transformers, "Transformer")
pypsa.io.import_components_from_dataframe(network, dclines, 'Link')
pypsa.io.import_components_from_dataframe(network, aclines, 'Line')

# Connect close buses (<= 1km distance)

In [ ]:
def connect_close_buses(network, radius=1.):
    adj = network.graph(["Line", "Transformer", "Link"]).adj
    
    n_lines_added = 0
    n_transformers_added = 0
    ac_buses = network.buses[network.buses.carrier == 'AC']

    for i,u in enumerate(ac_buses.index):

        vs = ac_buses[["x","y"]].iloc[i+1:]
        distance_km = pypsa.geo.haversine(vs, ac_buses.loc[u,["x","y"]])
    
        for j,v in enumerate(vs.index):
            km = distance_km[j,0]
        
            if km < radius:
                if u in adj[v]:
                    continue
                #print(u,v,ac_buses.at[u,"v_nom"], ac_buses.at[v,"v_nom"],km)
                
                if ac_buses.at[u,"v_nom"] != ac_buses.at[v,"v_nom"]:
                    network.add("Transformer","extra_trafo_{}_{}".format(u,v),s_nom=2000,bus0=u,bus1=v,x=0.1)
                    n_transformers_added += 1
                else:
                    network.add("Line","extra_line_{}_{}".format(u,v),s_nom=4000,bus0=u,bus1=v,x=0.1)
                    n_lines_added += 1
    
    print("Added {} lines and {} transformers to connect buses less than {} km apart."
          .format(n_lines_added, n_transformers_added, radius))

connect_close_buses(network)

## Remove all connected components with less than 10 buses

In [ ]:
network.determine_network_topology()

In [ ]:
sub_network_sizes = network.buses.groupby('sub_network').size()
subs_to_remove = sub_network_sizes.index[sub_network_sizes < 10]

In [ ]:
print("Removing {} small sub_networks (synchronous zones) with less than 10 buses. In total {} buses."
      .format(len(subs_to_remove), network.buses.sub_network.isin(subs_to_remove).sum()))

In [ ]:
network = network[~network.buses.sub_network.isin(subs_to_remove)]

In [ ]:
fig, ax = plt.subplots()
voltage_colors = {132.0: 'blue', 220.0: 'green', 300.0: 'orange', 380.0: 'red'}
network.plot(bus_sizes=5,
             bus_colors=network.buses['v_nom'].map(voltage_colors),
             line_colors=pd.concat(dict(Line=network.lines['v_nom'].map(voltage_colors),
                                        Link=pd.Series('violet', index=dclines.index))))

## Export network topology

In [ ]:
network.export_to_csv_folder('results/01-network-topology')

# Bus regions

In [ ]:
country_shapes = vshapes.countries(countries, minarea=0.1, tolerance=0.01, add_KV_to_RS=True)

In [ ]:
offshore_shapes = vshapes.eez(countries, tolerance=0.01)

On- and offshore area should be distributed to all buses

In [ ]:
vplot.shapes(offshore_shapes)
network.plot(bus_sizes=20)

In [ ]:
def buses_in_shape(shape):
    shape = shapely.prepared.prep(shape)
    return pd.Series(
        np.fromiter((shape.contains(Point(x, y))
                     for x, y in network.buses.loc[:,["x", "y"]].values),
                    dtype=bool, count=len(network.buses)),
        index=network.buses.index
    )

As there are multiple voltage levels and AC and DC buses, we preferable want to attach load and onshore generation to low voltage AC buses, while the offshore generation should go to high voltage AC buses, if there is no offshore bus built yet.

In [ ]:
network.buses['substation'] = network.buses.symbol.str.contains('substation', case=False)

def prefer_ac_voltage(x, which):
    index = x.index
    if len(index) == 1:
        return pd.Series(index, index)
    ac = x['carrier'] == 'AC'
    if ac.sum() > 0:
        x = x.loc[ac]
    key = (x.index[0]
           if x['v_nom'].isnull().all()
           else getattr(x['v_nom'], 'idx' + which)())
    return pd.Series(key, index)

gb = network.buses.loc[lambda df: df.substation].groupby(['x', 'y'], as_index=False, group_keys=False, sort=False)
bus_map_low = gb.apply(prefer_ac_voltage, 'min') #.reindex(network.buses.index)
buses_onshore_has_region_b = (bus_map_low == bus_map_low.index).reindex(network.buses.index, fill_value=False)
bus_map_high = gb.apply(prefer_ac_voltage, 'max')
buses_onshore_for_offshore_has_region_b = (bus_map_high == bus_map_high.index).reindex(network.buses.index, fill_value=False)

Split countries into voronoi cells and attach them to the buses as region_onshore and region_offshore entries

In [ ]:
%%time
network.buses.drop(['region_onshore', 'region_offshore'], inplace=True, axis=1, errors='ignore')

for country in countries:
    onshore_shape = country_shapes[country]

    # onshore
    buses_onshore_b = buses_in_shape(onshore_shape)
    network.buses.loc[buses_onshore_b, 'country'] = country     

    buses_onshore_locations = network.buses.loc[buses_onshore_b & buses_onshore_has_region_b, ["x", "y"]]
    network.buses.loc[buses_onshore_locations.index, 'region_onshore'] = voronoi_partition_pts(buses_onshore_locations.values, onshore_shape)

    # offshore
    if country not in offshore_shapes: continue
    offshore_shape = offshore_shapes[country]

    buses_offshore_b = buses_in_shape(offshore_shape)
    network.buses.loc[buses_offshore_b, 'country'] = country
    
    buses_offshore_locations = network.buses.loc[buses_offshore_b | (buses_onshore_b & buses_onshore_for_offshore_has_region_b), ["x", "y"]]
    buses_offshore_regions = pd.Series(voronoi_partition_pts(buses_offshore_locations.values, offshore_shape),
                                       index=buses_offshore_locations.index)
    buses_offshore_regions = buses_offshore_regions.loc[buses_offshore_regions.map(attrgetter('area')) > 1e-2]
    network.buses.loc[buses_offshore_regions.index, 'region_offshore'] = buses_offshore_regions

In [ ]:
onshorebuses_i = network.buses.index[network.buses['region_onshore'].notnull()]
offshorebuses_i = network.buses.index[network.buses['region_offshore'].notnull()]

Some buses don't lie in any country. Just add them to the nearest (hop-wise) country, but don't give them any load or generation, i.e. they will not be part of onshorebuses or offshorebuses.

In [ ]:
buses_nan_b = network.buses.country.isnull()
graph = network.graph()
network.buses.loc[buses_nan_b, 'country'] = \
    [(next(ifilter(len, map(lambda x: network.buses.loc[x, 'country'].dropna(),
                            BreadthFirstLevels(graph, [b]))))
      .value_counts().index[0])
     for b in network.buses.index[buses_nan_b]]

## Compute area of each region

In [ ]:
network.buses.drop(['area', 'area_offshore'], axis=1, inplace=True, errors='ignore')

on_b = network.buses['region_onshore'].notnull()
network.buses.loc[on_b, 'area'] = network.buses.loc[on_b, 'region_onshore'].map(vshapes.area)

off_b = network.buses['region_offshore'].notnull()
network.buses.loc[off_b, 'area_offshore'] = network.buses.loc[off_b, 'region_offshore'].map(vshapes.area)

## Illustrate regions of DE

In [ ]:
busesDE = network.buses.query('country == "DE"')

### Offshore

In [ ]:
plt.scatter(*busesDE.loc[busesDE.region_offshore.notnull(), ['x', 'y']].values.T).set_zorder(2)
vplot.shapes(busesDE.loc[busesDE.region_offshore.notnull(), 'region_offshore'])
vplot.shapes([offshore_shapes['DE']], outline=True, colour='r')
vplot.draw_basemap()

### Onshore

In [ ]:
plt.scatter(*busesDE.loc[busesDE.region_onshore.notnull(), ['x', 'y']].values.T).set_zorder(2)
vplot.shapes(busesDE.loc[busesDE.region_onshore.notnull(), 'region_onshore'])
vplot.draw_basemap()

# Generation

In [ ]:
network.set_snapshots(pd.date_range(start='2011', freq='h', periods=5*8760 + 24))

## Costs

In [ ]:
costs = vcostdata2.get_full_cost_CO2('diw2030', 0.)
costs

## Carriers

In [ ]:
network.carriers.drop(network.carriers.index, inplace=True)

carriers = pd.DataFrame([('onwind', 'WON'),
                        ('offwind', 'WOFF'),
                        ('solar', 'S'),
                        ('PHS', 'Hy'),
                        ('hydro', 'Hy'),
                        ('ror', 'Hy'),
                        #('H2', 'H'),
                        #('battery', 'B'),
                        ('OCGT', 'GO'),
                        ('CCGT', 'GC'),
                        ('coal', 'C'),
                        ('nuclear', 'N'),
                        ('lignite', 'L')],
                      columns=['name', 'short_name']).set_index('name')
carriers['co2_emissions'] = costs.loc[carriers.index, 'CO2intensity'].fillna(0.)

network.import_components_from_dataframe(carriers, "Carrier")

## Conventional Generation 

In [ ]:
powerplants = pm.MATCHED_dataset(subsume_uncommon_fueltypes=True, include_unavailables=True)

### Fuel types

Nearly all of the natural gas systems are CCGT

In [ ]:
conventionals = pd.DataFrame(powerplants.loc[(~powerplants.Fueltype.isin(['Wind', 'Hydro', 'Solar']))
                                               & powerplants.lon.notnull() & powerplants.lat.notnull()])

In [ ]:
conventionals.loc[conventionals.Fueltype == 'Natural Gas', 'Technology'].value_counts(dropna=False)

In [ ]:
conventionals.loc[(conventionals.Fueltype == 'Natural Gas') & conventionals.Technology.str.contains('OCGT|Open'), 'Fueltype'] = 'OCGT'
conventionals.loc[(conventionals.Fueltype == 'Natural Gas') & (~conventionals.Technology.str.contains('OCGT|Open').fillna(False)), 'Fueltype'] = 'CCGT'

In [ ]:
conventionals = conventionals.loc[conventionals.Fueltype.isin(['Hard Coal', 'Lignite', 'Nuclear', 'OCGT', 'CCGT', 'Hydro'])]

### Add costs, efficiency and a unique name

In [ ]:
conventionals.Fueltype.replace({'Hard Coal': 'coal', 'Nuclear': 'nuclear', 'Lignite': 'lignite'}, inplace=True)

In [ ]:
for ft in conventionals.Fueltype.unique():
    conv_ft_b = conventionals.Fueltype == ft
    conventionals.loc[conv_ft_b, 'marginal_cost'] = costs.at[ft, 'marginal']
    conventionals.loc[conv_ft_b, 'capital_cost'] = costs.at[ft, 'capital']
    conventionals.loc[conv_ft_b, 'efficiency'] = costs.at[ft, 'efficiency']
    conventionals.loc[conv_ft_b, 'name'] = carriers.at[ft, 'short_name'] + pd.RangeIndex(conv_ft_b.sum()).astype(str)

### Locate buses

In [ ]:
kdtree = sp.spatial.cKDTree(network.buses.loc[onshorebuses_i, ['x','y']].values)
conventionals.loc[:, 'bus'] = onshorebuses_i[kdtree.query(conventionals[['lon','lat']].values)[1]]

### Import into pypsa

In [ ]:
conventionals = conventionals.rename(columns={'Capacity': 'p_nom', 'Fueltype': 'carrier'})
conventionals = conventionals[['bus', 'name', 'p_nom', 'carrier', 'marginal_cost', 'capital_cost', 'efficiency']].set_index('name')

In [ ]:
network.import_components_from_dataframe(conventionals, 'Generator')

## Hydro Generation

In [ ]:
hydro = pd.DataFrame(powerplants.loc[lambda df: (df['Fueltype'] == 'Hydro') & df['lon'].notnull() & df['lat'].notnull()])
hydro.set_index(carriers.at['hydro', 'short_name'] + pd.RangeIndex(len(hydro)).astype(str), drop=False, inplace=True)

hydro.loc[:, 'bus'] = onshorebuses_i[kdtree.query(hydro[['lon', 'lat']].values)[1]]

In [ ]:
hydro = hydro.rename(columns={'Capacity':'p_nom', 'Technology':'technology'})
hydro = hydro.loc[hydro.technology.notnull(), ['bus', 'p_nom', 'technology']]

In [ ]:
hydro_capas = vhydro.get_hydro_capas()
hydro_capas.loc[hydro_capas['E_store[TWh]'] < 0.2, 'E_store[TWh]'] = 0.2

### Inflow

In [ ]:
def normed(x):
    return x/x.sum()

country_shapes = pd.Series(country_shapes)
country_shapes.index.rename('countries', inplace=True)
country_shapes = country_shapes.reindex(countries)

cutout_2011_2016 = atlite.Cutout('europe-2011-2016')
inflow = cutout_2011_2016.runoff(shapes=country_shapes,
                                 smooth=True,
                                 lower_threshold_quantile=True,
                                 normalize_using_yearly=vhydro.get_eia_annual_hydro_generation().reindex(columns=countries))

hydro_w_inflow = hydro.query("technology != 'Pumped Storage'")
hydro_cntry = hydro_w_inflow.bus.map(network.buses.country)

inflow_t = \
(inflow.sel(countries=hydro_cntry.values)
       .rename({'countries': 'name'})
       .assign_coords(name=hydro_w_inflow.index)
       .transpose('time', 'name')
       .to_pandas()
       .multiply(hydro_w_inflow.p_nom.groupby(hydro_cntry).transform(normed), axis=1)
       .divide(hydro_w_inflow.p_nom, axis=1))

### RoR

In [ ]:
hydro_ror = hydro.loc[hydro.technology == 'Run-Of-River']

hydro_ror = hydro_ror.assign(
    carrier="ror",
    efficiency=costs.at['ror', 'efficiency'],
    weight=hydro_ror['p_nom']
)

pypsa.io.import_components_from_dataframe(network, hydro_ror, 'Generator')

rorpotential = inflow_t.loc[:,hydro_ror.index].where(lambda df: df<=1., other=1.)
pypsa.io.import_series_from_dataframe(network, rorpotential, 'Generator', 'p_max_pu')

### PHS

In [ ]:
hydro_phs = hydro.loc[hydro.technology == 'Pumped Storage']

hydro_phs = hydro_phs.assign(
    carrier='PHS',
    max_hours=6,
    p_max_pu=1.,
    p_min_pu=-1.,
    efficiency_store=np.sqrt(costs.at['PHS','efficiency']),
    efficiency_dispatch=np.sqrt(costs.at['PHS','efficiency']),
    cyclic_state_of_charge=True
)

network.import_components_from_dataframe(hydro_phs, 'StorageUnit')

### Reservoir

In [ ]:
hydro_reservoir = hydro.loc[hydro.technology == 'Reservoir']

hydro_reservoir_max_hours = hydro_capas['E_store[TWh]']*1e6/hydro_reservoir.p_nom.groupby(hydro_cntry).sum()

hydro_reservoir = hydro_reservoir.assign(
    carrier="hydro",
    max_hours=hydro_cntry.loc[hydro_reservoir.index].map(hydro_reservoir_max_hours),
    p_max_pu=1.,  #dispatch
    p_min_pu=0.,
    efficiency_dispatch=costs.at['hydro','efficiency'],
    efficiency_store=0,
    cyclic_state_of_charge=True,
)

network.import_components_from_dataframe(hydro_reservoir, 'StorageUnit')

reservoirpotential = inflow_t.loc[:,hydro_reservoir.index]
pypsa.io.import_series_from_dataframe(network, reservoirpotential, 'StorageUnit', 'inflow')

## Renewables

In [ ]:
cutout = atlite.Cutout('europe-2011-2016')

### Landuse potentials

In [ ]:
%%time
onshorepotentials = xr.DataArray(vlanduse.windonshorepotentials(cutout, cushion_factor=0.3, distance=1000),
                                 [cutout.coords['y'], cutout.coords['x']])
offshorepotentials = xr.DataArray(vlanduse.windoffshorepotentials(cutout, cushion_factor=0.3),
                                  [cutout.coords['y'], cutout.coords['x']])
solarpotentials = xr.DataArray(vlanduse.solarpotentials(cutout),
                               [cutout.coords['y'], cutout.coords['x']])

### Indicatormatrices and capacities

In [ ]:
onshoreindicatormatrix = cutout.indicatormatrix(network.buses.loc[onshorebuses_i, 'region_onshore'])

In [ ]:
offshoreindicatormatrix = cutout.indicatormatrix(network.buses.loc[offshorebuses_i, 'region_offshore'])

In [ ]:
# Remove raster cells from indicatormatrix with very low potential (incl. those beyond 50m depth)
ocean_depth_cutoff = 50
offshorepotentials.values[(cutout.meta['height'] < - ocean_depth_cutoff).transpose(*offshorepotentials.dims)] = 0.

offshoreindicatormatrix[:,(offshorepotentials < 1e-4).stack(spatial=('y', 'x')).values] = 0.

keep_offshore_b = np.asarray(offshoreindicatormatrix.sum(axis=1)).squeeze() > 0.
network.buses.loc[offshorebuses_i[~keep_offshore_b], 'region_offshore'] = np.nan
network.buses.loc[offshorebuses_i[~keep_offshore_b], 'area_offshore'] = np.nan
offshoreindicatormatrix = offshoreindicatormatrix[keep_offshore_b]
offshorebuses_i = offshorebuses_i[keep_offshore_b]

In [ ]:
onshore_i = ('WON' + pd.RangeIndex(len(onshorebuses_i)).astype(str)).rename("name")
offshore_i = ('WOF' + pd.RangeIndex(len(offshorebuses_i)).astype(str)).rename("name")
solar_i = ('S' + pd.RangeIndex(len(onshorebuses_i)).astype(str)).rename("name")

### Generation time-series

In [ ]:
onshorewindturbine ='Vestas_V112_3MW'
offshorewindturbine='NREL_ReferenceTurbine_5MW_offshore'
solarpanel = 'KANENA'
orientation='latitude_optimal'

In [ ]:
%%time
onshorewindcapacityfactor = cutout.wind(turbine=onshorewindturbine, smooth=True, capacity_factor=True)
onshorewindlayout = onshorewindcapacityfactor * onshorepotentials
onshore, onshore_weights = cutout.wind(turbine=onshorewindturbine, smooth=True,
                                       matrix=onshoreindicatormatrix, index=onshore_i,
                                       layout=onshorewindlayout,
                                       per_unit=True, return_units=True)

In [ ]:
%%time
offshorewindcapacityfactor = cutout.wind(turbine=offshorewindturbine, smooth=True, capacity_factor=True)
offshorewindlayout = offshorewindcapacityfactor * offshorepotentials
offshore, offshore_weights = cutout.wind(turbine=offshorewindturbine, smooth=True,
                                         matrix=offshoreindicatormatrix, index=offshore_i,
                                         layout=offshorewindlayout,
                                         per_unit=True, return_units=True)

In [ ]:
%%time
solarlayout = cutout.pv(panel=solarpanel, orientation=orientation, capacity_factor=True) * solarpotentials
solar, solar_weights = cutout.pv(panel=solarpanel, orientation=orientation,
                                 matrix=onshoreindicatormatrix, index=solar_i,
                                 layout=solarlayout,
                                 per_unit=True, return_units=True)

# solar capacity factors in REatlas are too high compared to other
# studies; we correct this by applying a constant 'inverter
# inefficiency' to the p_max_pu timeseries; comparing with
# Pietzker+(2014) http://dx.doi.org/10.1016/j.apenergy.2014.08.011
# the produced-power weighted average of capacity factor ratios is 1.2683
solar /= 1.2683

### Determine maximal capacity per Voronoi cell

In [ ]:
def p_nom_max(potential, layout, indicatormatrix, weights):
    weights = weights.to_series()
    relativepotential = (potential / layout).stack(spatial=('y', 'x')).values
    return pd.Series([np.nanmin(relativepotential[row.nonzero()[1]])
                      for row in indicatormatrix.tocsr()],
                     index=weights.index) * weights

In [ ]:
onshore_p_nom_max = p_nom_max(onshorepotentials, onshorewindlayout, onshoreindicatormatrix, onshore_weights)
offshore_p_nom_max = p_nom_max(offshorepotentials, offshorewindlayout, offshoreindicatormatrix, offshore_weights)
solar_p_nom_max = p_nom_max(solarpotentials, solarlayout, onshoreindicatormatrix, solar_weights)

### Illustrate German capacity factors and available capacity

In [ ]:
onshore_de = pd.Series(onshorebuses_i, onshore_i).map(network.buses.country) == 'DE'
offshore_de = pd.Series(offshorebuses_i, offshore_i).map(network.buses.country) == 'DE'
solar_de = pd.Series(onshorebuses_i, solar_i).map(network.buses.country) == 'DE'
bins = np.r_[0.05:0.5:0.01]
#bins = np.r_[0.08:0.15:0.005]
plt.figure()
pd.concat(
    dict(onshore=onshore_p_nom_max.loc[onshore_de].groupby(pd.cut(onshore.mean('time').to_series().loc[onshore_de], bins)).sum(),
         offshore=offshore_p_nom_max.loc[offshore_de].groupby(pd.cut(offshore.mean('time').to_series().loc[offshore_de], bins)).sum(),
         solar=solar_p_nom_max.loc[solar_de].groupby(pd.cut(solar.mean('time').to_series().loc[solar_de], bins)).sum()),
    axis=1
).fillna(0.).pipe(lambda df: df/1e3).plot(marker='o', rot=90)
plt.ylabel("Max Capacity / GW")
plt.tight_layout()

if False:
    plt.savefig('capacityfactors_de.pdf')
#plt.savefig('capacityfactors_solar_de.pdf')

### Plot onshore and offshore potentials and voronoi cells

In [ ]:
grid_cells = pd.Series(cutout.grid_cells())
cell_areas = vlanduse._cutout_cell_areas(cutout)
onshorede = np.where(network.buses.loc[onshorebuses_i, 'country'] == 'DE')
offshorede = np.where(network.buses.loc[offshorebuses_i, 'country'] == 'DE')

In [ ]:
onlayout = pd.Series((onshorewindlayout.transpose('y', 'x').values / cell_areas * 8760 / 1e3).ravel())
offlayout = pd.Series((offshorewindlayout.transpose('y', 'x').values / cell_areas * 8760 / 1e3).ravel())
solarlayout = pd.Series((solarlayout.transpose('y', 'x').values / cell_areas * 8760 / 1e3).ravel())

In [ ]:
# onbuses_i = np.concatenate([onshoreindicatormatrix[busno].nonzero()[1]
#                                                 for busno in busnos]
windpotential_per_grid_cell = (
    onlayout * pd.Series(np.asarray(onshoreindicatormatrix[onshorede].sum(axis=0)).squeeze())
    + offlayout * pd.Series(np.asarray(offshoreindicatormatrix[offshorede].sum(axis=0)).squeeze())
).loc[lambda s: s>0]

In [ ]:
solarpotential_per_grid_cell = (
    solarlayout * pd.Series(np.asarray(onshoreindicatormatrix[onshorede].sum(axis=0)).squeeze())
).loc[lambda s: s>0]

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

ax.set_title('DE Wind average power density [GWh/a/km$^2$]')

sh = vplot.shapes(grid_cells, windpotential_per_grid_cell, cmap='Blues',
                  norm=mpl.colors.Normalize(vmax=10), ax=ax)
vplot.shapes(network.buses.loc[onshorebuses_i[onshorede], 'region_onshore'],
             outline=True, colour='black', linestyle='-', ax=ax)
vplot.shapes(network.buses.loc[offshorebuses_i[offshorede], 'region_offshore'],
             outline=True, colour='black', linestyle='-', ax=ax)
fig.colorbar(sh, extend='max')

vplot.draw_basemap(resolution='h', ax=ax)
for c in ax.collections[-2:]:
    c.set_rasterized(True)

if False:
    fig.savefig('wind-avg-power-density.pdf', transparent=True, bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

ax.set_title('DE Solar average power density [GWh/a/km$^2$]')

sh = vplot.shapes(grid_cells, solarpotential_per_grid_cell, cmap='YlOrBr', ax=ax)
vplot.shapes(network.buses.loc[onshorebuses_i[onshorede], 'region_onshore'],
             outline=True, colour='black', linestyle='-', ax=ax)
vplot.shapes(network.buses.loc[offshorebuses_i[offshorede], 'region_offshore'],
             outline=True, colour='black', linestyle='-', ax=ax)
fig.colorbar(sh)

vplot.draw_basemap(resolution='h')
for c in ax.collections[-2:]:
    c.set_rasterized(True)
    
if False:
    fig.savefig('solar-avg-power-density.pdf', transparent=True, bbox_inches='tight')

### Add renewable generators to PyPSA model

In [ ]:
onshorewindturbines = pd.DataFrame(dict(p_nom_extendable=True,
                                        p_nom_max=onshore_p_nom_max,
                                        bus=onshorebuses_i,
                                        carrier='onwind',
                                        marginal_cost=costs.at['onwind', 'marginal'],
                                        capital_cost=costs.at['onwind', 'capital'],
                                        weight=onshore_weights),
                                   index=onshore_i)
pypsa.io.import_components_from_dataframe(network, onshorewindturbines, 'Generator')
pypsa.io.import_series_from_dataframe(
    network,
    onshore.sel(time=network.snapshots).transpose('time', 'name').to_pandas(),
    'Generator', 'p_max_pu'
)

In [ ]:
offshorewindturbines = pd.DataFrame(dict(p_nom_extendable=True,
                                         p_nom_max=offshore_p_nom_max,
                                         bus=offshorebuses_i,
                                         carrier='offwind',
                                         marginal_cost=costs.at['offwind', 'marginal'],
                                         capital_cost=costs.at['offwind', 'capital'],
                                         weight=offshore_weights),
                                   index=offshore_i)
pypsa.io.import_components_from_dataframe(network, offshorewindturbines, 'Generator')
pypsa.io.import_series_from_dataframe(
    network,
    offshore.sel(time=network.snapshots).transpose('time', 'name').to_pandas(),
    'Generator', 'p_max_pu'
)

In [ ]:
solarpanels = pd.DataFrame(dict(p_nom_extendable=True,
                                p_nom_max=solar_p_nom_max,
                                bus=onshorebuses_i,
                                carrier='solar',
                                marginal_cost=costs.at['solar', 'marginal'],
                                capital_cost=costs.at['solar', 'capital'],
                                weight=solar_weights),
                            index=solar_i)
pypsa.io.import_components_from_dataframe(network, solarpanels, 'Generator')
pypsa.io.import_series_from_dataframe(
    network,
    solar.sel(time=network.snapshots).transpose('time', 'name').to_pandas(),
    'Generator', 'p_max_pu'
)

# Load

In [ ]:
from vresutils import load as vload

In [ ]:
%%time
loads = vload.timeseries_shapes(network.buses.loc[onshorebuses_i, 'region_onshore'],
                                network.buses.loc[onshorebuses_i, 'country'])

In [ ]:
pypsa.io.import_components_from_dataframe(network, pd.DataFrame(dict(bus=onshorebuses_i), index=onshorebuses_i), "Load")

In [ ]:
pypsa.io.import_series_from_dataframe(network, loads, 'Load', 'p_set')

In [ ]:
# Fill up the one hour
network.loads_t.p_set.ffill(inplace=True)

# Export model

In [ ]:
region_onshore = gpd.GeoDataFrame(dict(geometry=network.buses.region_onshore)).dropna().reset_index()
region_offshore = gpd.GeoDataFrame(dict(geometry=network.buses.region_offshore)).dropna().reset_index()

def export_as_geojson(df, fn):
    if os.path.exists(fn):
        os.unlink(fn)
    df.to_file(fn, driver='GeoJSON')

export_as_geojson(region_onshore, 'results/region_onshore.json')
export_as_geojson(region_offshore, 'results/region_offshore.json')

In [ ]:
network.buses['region_onshore'] =  gpd.read_file('results/region_onshore.json').set_index('name')['geometry']
network.buses['region_offshore'] = gpd.read_file('results/region_offshore.json').set_index('name')['geometry']

In [ ]:
network.buses.drop(['region_onshore', 'region_offshore'], axis=1, inplace=True)
network.export_to_csv_folder("results/02-pypsa-europe")